# Propagation in Naturalis

## Orbital States

To propagate orbital states in Naturalis, use the following imports from the `naturalis.dynamics.propagator` module.


In [1]:
from naturalis.dynamics.propagator import OrbitalPropagator, ForceModel, PropagationMethod

The propagator takes in a list of `ForceModels` and a `PropagationMethod`. Here we will only use a single force, the central two-body force, and we will use the `RK45` method.

In [2]:
propagator = OrbitalPropagator(
    forces=[ForceModel.TWO_BODY],
    method=PropagationMethod.RK45
)

Next, we import Earth's specific gravitaty parameter `MU` from `naturalis.constants` and the `OrbitalState` class from `naturalis.dynamics.orbit`. Since `position` and `velocity` are represented with `numpy` arrays, we will also need to import that. We will define an initial state that marks the start of propagation, which represents a circular Low-Earth orbit.

In [3]:
from naturalis.dynamics.orbit import OrbitalState
from naturalis.constants import MU

import numpy as np

initial_state = OrbitalState(
    mu=MU,
    time=0,
    position=np.array([7000.0, 0.0, 0.0]),
    velocity=np.array([0.0, 7.546, 0.0])
)

Finally, we can propagate the state forward with the `propagate_state_by_time` function. This propagates by a relative time (initial state time plus given time), but we could also plot to an absolute time with `propagate_state_to_time`.

In [4]:
final_state: OrbitalState = propagator.propagate_state_by_time(
    state=initial_state,
    time=1000.0
)

We can print out the representation to see that we have indeed propagated the state.

In [5]:
final_state

OrbitalState(mu=398600.4418, time=np.float64(1000.0), position=array([3311.58038824, 6167.06623909,    0.        ]), velocity=array([-6.64824078,  3.56985407,  0.        ]))

To view our states graphically, we can import `plot_orbital_states` from `naturalis.plotting`. We must pass in a list of states and a `plotly.graph_objects.Figure` object. The latter is done so you can stack graphs on a single figure. Then we add the Earth to the plot, setting `atmosphere` to `True` to plot that as well. Remember to use the `show` function in order to see your plots.

In [6]:
from naturalis.plotting import plot_orbital_states, plot_earth
import plotly.graph_objects as go

fig = go.Figure()

plot_earth(
    figure=fig, 
    atmosphere=True
)

plot_orbital_states(
    figure=fig,
    states=[initial_state, final_state]
)

fig.show()

Now we can clearly see that state has been propagated! Hover over each state to see more information about it.

## Segments

In addition to individual states, we can have a collection of states that represent a natural motion segment. To do this, we can propagate and initial state as we have done before, but this time using the `propagate_state_to_segment` or `propagate_state_by_segment` methods, which use absolute and relative time, respectively. Both methods return a `Segment` object, which is just an initial and final state.

In [7]:
from naturalis.dynamics.orbit import Segment

segment: Segment = propagator.propagate_state_by_segment(
    initial_state=initial_state,
    time=1000.0
)

segment

Segment(initial_state=OrbitalState(mu=398600.4418, time=0, position=array([7000.,    0.,    0.]), velocity=array([0.   , 7.546, 0.   ])), final_state=OrbitalState(mu=398600.4418, time=np.float64(1000.0), position=array([3311.58038824, 6167.06623909,    0.        ]), velocity=array([-6.64824078,  3.56985407,  0.        ])))

We can plot this, too! The `plot_segments` function takes in a `Figure`, a list of `Segments`, and the `OrbitalPropagator` you used to make the segments. This is need because we only store the terminal states on the segment, so to recreate it we must re-propagate it.

In [8]:
from naturalis.plotting import plot_segments

plot_segments(
    figure=fig, 
    segments=[segment],
    propagator=propagator
)

fig.show()

## Trajectories

The final orbital data type is the `Trajectory`, which is a collection of natural-motion `Segments` and the set of `Burns` that takes us between them. The easiest way to get a trajectory is to use the `propagate_state_to_trajectory` method on the `OrbitalPropagator` class, which takes in an initial `OrbitalState` and a list of `Burns`. Make sure to import the `Burn` class from `naturalis.dynamics.orbit`. The `position` parameter on the burn is not needed here, as the propagator will solve for that.

In [9]:
from naturalis.dynamics.orbit import Burn

burns = [Burn(500.0, np.array([0.0, 1.0, 0.0])), Burn(1500.0, np.array([-1.0, 0.0, 0.0]))]

In [10]:
from naturalis.dynamics.orbit import Trajectory

trajectory: Trajectory = propagator.propagate_state_to_trajectory(
    initial_state=initial_state,
    burns=burns,
    time=17000.0
)

trajectory

Trajectory(segments=[Segment(initial_state=OrbitalState(mu=398600.4418, time=np.float64(500.0), position=array([6007.54207804, 3592.94335257,    0.        ]), velocity=array([-3.8732554 ,  7.47612822,  0.        ])), final_state=OrbitalState(mu=398600.4418, time=np.float64(1500.0), position=array([-129.27560668, 8232.75230307,    0.        ]), velocity=array([-8.17020801,  1.55485204,  0.        ])))], burns=[Burn(time=500.0, delta_v=array([0., 1., 0.]), position=array([6007.54207804, 3592.94335257,    0.        ])), Burn(time=1500.0, delta_v=array([-1.,  0.,  0.]), position=array([-129.27560668, 8232.75230307,    0.        ]))], initial_coast=Segment(initial_state=OrbitalState(mu=398600.4418, time=0, position=array([7000.,    0.,    0.]), velocity=array([0.   , 7.546, 0.   ])), final_state=OrbitalState(mu=398600.4418, time=np.float64(500.0), position=array([6007.54207804, 3592.94335257,    0.        ]), velocity=array([-3.8732554 ,  7.47612822,  0.        ]))), final_coast=Segment(ini

In [11]:
from naturalis.plotting import plot_trajectory

trajectory_fig = go.Figure()

plot_earth(figure=trajectory_fig, atmosphere=True)
plot_trajectory(
    figure=trajectory_fig,
    trajectory=trajectory,
    propagator=propagator,
    plot_title=f"Trajectory Plot - {trajectory.total_delta_v:.3f} km/s"
)

trajectory_fig.show()

# Trajectory Optimization in Naturalis

Now that we've gone over all the orbital dynamics, we can start optimizing trajectories!

In [12]:

from naturalis.solvers.lambert import LambertSolverType, IzzoLambertSolver, LambertSolution

In [13]:
solver = IzzoLambertSolver(
    mu=MU
)

In [14]:
final_state = OrbitalState(
    mu=MU,
    time=60000.0,
    position=np.array([0.0, 42164.0, 0.0]),
    velocity=np.array([3.07, 0.0,  0.0])
)

In [15]:
solution: LambertSolution = solver.solve(
    initial_state,
    final_state
)

In [16]:
from naturalis.plotting import plot_solution

solution_fig = go.Figure()

plot_earth(
    figure=solution_fig,
    atmosphere=True
)

plot_solution(
    figure=solution_fig,
    solution=solution,
    propagator=propagator
)

solution_fig.show()

In [17]:
from naturalis.plotting import plot_primer_trajectory
from naturalis.mathematics.primer import PrimerVectorAnalyzer

primer_analyzer = PrimerVectorAnalyzer(
    mu=MU
)

primer_trajectory = primer_analyzer.analyze_trajectory(
    solution.trajectory
)

primer_figure = go.Figure()

plot_primer_trajectory(
    figure=primer_figure,
    trajectory=primer_trajectory
)

primer_figure.show()

In [18]:
from naturalis.solvers.n_burn import NBurnSolver, NBurnSolution

In [19]:
solver = NBurnSolver(
    mu=MU,
    propagator=propagator,
    lambert=LambertSolverType.IZZO # most robust Lambert solver
)

In [20]:
solution: NBurnSolution = solver.solve(
    initial_state=initial_state,
    final_state=final_state,
    max_burns=6
)

In [21]:
solution_fig = go.Figure()

plot_earth(
    figure=solution_fig,
    atmosphere=True
)

plot_solution(
    figure=solution_fig,
    solution=solution,
    propagator=propagator
)

solution_fig.show()

In [22]:
primer_trajectory = primer_analyzer.analyze_trajectory(
    solution.trajectory
)

primer_figure = go.Figure()

plot_primer_trajectory(
    figure=primer_figure,
    trajectory=primer_trajectory
)

primer_figure.show()